In [ ]:
import  numpy as np
from PIL import Image
import cv2
import math
import sys
from pathlib import Path

root_path = Path.cwd().parent
if str(root_path) not in sys.path:
    sys.path.append(str(root_path))

import Implementation.ascon as ascon

In [ ]:
Path("ASCON Encryption Files").mkdir(parents=True, exist_ok=True)

img_path = r"Enter the path of the input image"
image = Image.open(img_path)
original_image = image.resize((256, 256)) 
original_image.save("ASCON Encryption Files/original_image.png")
original_image = cv2.imread("ASCON Encryption Files/original_image.png", cv2.IMREAD_GRAYSCALE)
image_array = np.array(original_image)

ciphertext = ascon.demo_aead("Ascon-128", image_array)
hexa_string = ciphertext[:-16].hex()
ciphertext_decrypt = np.array([int(hexa_string[i:i+2], 16) for i in range(0, len(hexa_string), 2)], dtype=np.uint8)
encrypted_image_array = ciphertext_decrypt.reshape(image_array.shape)
encrypted_image = Image.fromarray(encrypted_image_array)
encrypted_image.save("ASCON Encryption Files/encrypted_image.png")
encrypted_image = cv2.imread("ASCON Encryption Files/encrypted_image.png", cv2.IMREAD_GRAYSCALE)
encrypted_image_path = "ASCON Encryption Files/encrypted_image.png"

In [ ]:
# Unified function to calculate metrics
def calculate_metrics(original_image, encrypted_image):
    if original_image is None:
        print("Error: Unable to load original image.")
        return

    # NPCR
    num_changed_pixels = np.sum(original_image != encrypted_image)
    total_pixels = original_image.size
    npcr = (num_changed_pixels / total_pixels) * 100

    # UACI
    abs_diff = np.abs(original_image - encrypted_image)
    uaci = np.mean(abs_diff / 255) * 100

    # Mean Deviation
    mean_deviation = np.mean(np.abs(original_image - encrypted_image))

    # MSE
    mse = np.mean((original_image - encrypted_image) ** 2)

    # PSNR
    max_intensity = 255
    psnr = 10 * np.log10(max_intensity ** 2 / mse)

    with open(encrypted_image_path, 'rb') as encrypted_image_file:
        encrypted_data = encrypted_image_file.read()

    # Entropy
    byte_frequency = [0] * 256
    for byte in encrypted_data:
        byte_frequency[byte] += 1

    entropy = 0
    total_bytes = len(encrypted_data)
    for freq in byte_frequency:
        if freq != 0:
            probability = freq / total_bytes
            entropy -= probability * math.log2(probability)

    return npcr, uaci, mean_deviation, mse, psnr, entropy

In [ ]:
# Calculate metrics
npcr, uaci, mean_deviation, mse, psnr, entropy = calculate_metrics(original_image, encrypted_image)

# Display metrics
print("NPCR score:", npcr)
print("UACI score:", uaci)
print("Mean Deviation:", mean_deviation)
print("Mean Squared Error (MSE):", mse)
print("Peak Signal-to-Noise Ratio (PSNR):", psnr)
print("Entropy of the encrypted image data:", entropy)